In [4]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("PySparkChallenge100k")
    .master("local[*]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "false")
    .getOrCreate())

players = spark.read.csv(
    "/mnt/c/Users/33687/Desktop/pyspark_challenge_100k/players_100k.csv",
    header=True,
    inferSchema=True
)

stats = spark.read.csv(
    "/mnt/c/Users/33687/Desktop/pyspark_challenge_100k/stats_100k.csv",
    header=True,
    inferSchema=True
)

players.show()


+---------+---------+--------+--------+---------+
|Player_ID|   Player|    Team|Position|   Salary|
+---------+---------+--------+--------+---------+
|        1| Player_1|    NULL|      SF|4190611.0|
|        2| Player_2|    Heat|      SG|3050658.0|
|        3| Player_3|    Suns|      SF|4519194.0|
|        4| Player_4|Clippers|      SF|7000660.0|
|        5| Player_5|   Spurs|      PF|2907846.0|
|        6| Player_6|  Lakers|      PG|2907870.0|
|        7| Player_7|    Nets|      SF|7200107.0|
|        8| Player_8|    Nets|       C|     NULL|
|        9| Player_9|Clippers|      SF|2585069.0|
|       10|Player_10|Warriors|       C|4287781.0|
|       11|Player_11|    Nets|      PG|2592909.0|
|       12|Player_12|   Bulls|      SF|2589914.0|
|       13|Player_13|Warriors|      SG|3689425.0|
|       14|Player_14|  Lakers|      SF|1255896.0|
|       15|Player_15|    NULL|      SG|     NULL|
|       16|Player_16|   Spurs|      PG|2467846.0|
|       17|Player_17|    Heat|      SF|1970079.0|


In [5]:
players.show()

+---------+---------+--------+--------+---------+
|Player_ID|   Player|    Team|Position|   Salary|
+---------+---------+--------+--------+---------+
|        1| Player_1|    NULL|      SF|4190611.0|
|        2| Player_2|    Heat|      SG|3050658.0|
|        3| Player_3|    Suns|      SF|4519194.0|
|        4| Player_4|Clippers|      SF|7000660.0|
|        5| Player_5|   Spurs|      PF|2907846.0|
|        6| Player_6|  Lakers|      PG|2907870.0|
|        7| Player_7|    Nets|      SF|7200107.0|
|        8| Player_8|    Nets|       C|     NULL|
|        9| Player_9|Clippers|      SF|2585069.0|
|       10|Player_10|Warriors|       C|4287781.0|
|       11|Player_11|    Nets|      PG|2592909.0|
|       12|Player_12|   Bulls|      SF|2589914.0|
|       13|Player_13|Warriors|      SG|3689425.0|
|       14|Player_14|  Lakers|      SF|1255896.0|
|       15|Player_15|    NULL|      SG|     NULL|
|       16|Player_16|   Spurs|      PG|2467846.0|
|       17|Player_17|    Heat|      SF|1970079.0|


In [7]:
#Compte le nombre total de lignes dans chaque fichier.
print('stats ', stats.count())
print('players ', players.count())

stats  100000
players  100000


In [8]:
# Affiche le schéma (printSchema) des deux DataFrames
stats.printSchema()
players.printSchema()

root
 |-- Player_ID: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- GP: integer (nullable = true)
 |-- BLK: double (nullable = true)
 |-- DRB: double (nullable = true)
 |-- STL: double (nullable = true)
 |-- PTS: double (nullable = true)

root
 |-- Player_ID: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Salary: double (nullable = true)



In [15]:
from pyspark.sql.functions import col

In [17]:
# Détecte les colonnes contenant des valeurs NULL dans players et stats
players.filter( col('Player').isNull() | col('Team').isNull() | col('Position').isNull() | col('Salary').isNull() ).show()

+---------+----------+--------+--------+---------+
|Player_ID|    Player|    Team|Position|   Salary|
+---------+----------+--------+--------+---------+
|        1|  Player_1|    NULL|      SF|4190611.0|
|        8|  Player_8|    Nets|       C|     NULL|
|       15| Player_15|    NULL|      SG|     NULL|
|       34| Player_34|   76ers|      PG|     NULL|
|       40| Player_40|    NULL|      SG|3607157.0|
|       49| Player_49|   Bucks|      PF|     NULL|
|       50| Player_50|    NULL|      PG|1353873.0|
|       56| Player_56|    NULL|       C|5207654.0|
|       59| Player_59|  Knicks|      PG|     NULL|
|       64| Player_64|    NULL|      SG|1797481.0|
|       68| Player_68| Raptors|      SG|     NULL|
|       79| Player_79|Warriors|      SF|     NULL|
|       82| Player_82|   Spurs|      PF|     NULL|
|       83| Player_83|    NULL|      PG|6844439.0|
|       88| Player_88| Raptors|      PF|     NULL|
|       89| Player_89|    NULL|      PG|2508310.0|
|      102|Player_102|    NULL|

In [18]:
stats.show()

+---------+------+---+----+----+----+----+
|Player_ID|Season| GP| BLK| DRB| STL| PTS|
+---------+------+---+----+----+----+----+
|        1|  2023| 20| 2.6| 5.5| 2.0|27.5|
|        2|  2023| 41|NULL| 7.1| 0.8|34.1|
|        3|  2023| 30|NULL| 2.4| 2.8|NULL|
|        4|  2022| 16| 1.7|10.5| 0.4|NULL|
|        5|  2023| 34| 1.3| 2.8| 0.2| 3.0|
|        6|  2024| 56| 1.0| 8.0| 2.4|36.8|
|        7|  2023| 22| 2.9| 4.5|NULL|18.3|
|        8|  2023| 43| 2.6| 3.4| 0.8|13.0|
|        9|  2023| 76| 2.5| 6.3| 1.2|22.7|
|       10|  2023| 44| 2.0| 1.5| 0.9| 4.0|
|       11|  2023| 72| 3.2| 3.1| 1.0|15.4|
|       12|  2023| 77| 3.3| 4.9| 0.6|34.2|
|       13|  2023| 46| 3.4| 9.2| 1.6| 4.8|
|       14|  2023| 67| 2.6|10.2| 2.9| 2.2|
|       15|  2023| 71| 2.4|10.5| 2.1|13.0|
|       16|  2023| 19| 2.8| 3.6| 0.8|36.1|
|       17|  2023| 44| 0.8|11.0| 1.2|NULL|
|       18|  2023| 22| 2.8|11.4| 1.9| 2.5|
|       19|  2024| 39| 1.4|NULL| 2.9|19.6|
|       20|  2023| 64| 1.7| 5.8| 1.4| 6.7|
+---------+

In [19]:
stats.filter( col('Season').isNull() | col('GP').isNull() | col('BLK').isNull() | col('DRB').isNull() | col('STL').isNull() | col('PTS').isNull() ).show()

+---------+------+---+----+----+----+----+
|Player_ID|Season| GP| BLK| DRB| STL| PTS|
+---------+------+---+----+----+----+----+
|        2|  2023| 41|NULL| 7.1| 0.8|34.1|
|        3|  2023| 30|NULL| 2.4| 2.8|NULL|
|        4|  2022| 16| 1.7|10.5| 0.4|NULL|
|        7|  2023| 22| 2.9| 4.5|NULL|18.3|
|       17|  2023| 44| 0.8|11.0| 1.2|NULL|
|       19|  2024| 39| 1.4|NULL| 2.9|19.6|
|       28|  2023| 62| 0.5|NULL|NULL|21.4|
|       29|  2023| 36| 0.1|10.4| 0.5|NULL|
|       31|  2023| 55| 0.1|NULL| 2.0|22.6|
|       35|  2022| 59| 2.3|NULL| 2.8|37.1|
|       36|  2023| 41| 1.5|10.5|NULL|NULL|
|       37|  2023| 33|NULL|10.0| 0.6| 3.5|
|       42|  2022| 40| 0.4| 8.1|NULL|34.9|
|       45|  2022| 56| 0.6|NULL| 2.1|27.2|
|       60|  2023| 51| 0.3| 7.6| 0.6|NULL|
|       65|  2022| 49|NULL| 1.1|NULL|16.2|
|       66|  2023| 66|NULL| 1.9| 1.4|15.7|
|       68|  2023| 59| 0.5|NULL| 1.9|17.3|
|       77|  2024| 47| 3.2|NULL| 1.6|30.8|
|       82|  2023| 80|NULL|10.5| 1.4|26.8|
+---------+

In [22]:
# Remplace les NULL dans Salary par 0.
# Remplace les NULL dans Team par "Unknown".
# Remplace les NULL dans les stats numériques (PTS, BLK, DRB, STL) par 0.
players.fillna( { 'Salary':0 , 'Team' : "Unknown" }  ).show()

+---------+---------+--------+--------+---------+
|Player_ID|   Player|    Team|Position|   Salary|
+---------+---------+--------+--------+---------+
|        1| Player_1| Unknown|      SF|4190611.0|
|        2| Player_2|    Heat|      SG|3050658.0|
|        3| Player_3|    Suns|      SF|4519194.0|
|        4| Player_4|Clippers|      SF|7000660.0|
|        5| Player_5|   Spurs|      PF|2907846.0|
|        6| Player_6|  Lakers|      PG|2907870.0|
|        7| Player_7|    Nets|      SF|7200107.0|
|        8| Player_8|    Nets|       C|      0.0|
|        9| Player_9|Clippers|      SF|2585069.0|
|       10|Player_10|Warriors|       C|4287781.0|
|       11|Player_11|    Nets|      PG|2592909.0|
|       12|Player_12|   Bulls|      SF|2589914.0|
|       13|Player_13|Warriors|      SG|3689425.0|
|       14|Player_14|  Lakers|      SF|1255896.0|
|       15|Player_15| Unknown|      SG|      0.0|
|       16|Player_16|   Spurs|      PG|2467846.0|
|       17|Player_17|    Heat|      SF|1970079.0|


In [23]:
players = players.fillna( { 'Salary':0 , 'Team' : "Unknown" }  )

In [25]:
stats = stats.fillna(0)

In [26]:
stats.show()

+---------+------+---+---+----+---+----+
|Player_ID|Season| GP|BLK| DRB|STL| PTS|
+---------+------+---+---+----+---+----+
|        1|  2023| 20|2.6| 5.5|2.0|27.5|
|        2|  2023| 41|0.0| 7.1|0.8|34.1|
|        3|  2023| 30|0.0| 2.4|2.8| 0.0|
|        4|  2022| 16|1.7|10.5|0.4| 0.0|
|        5|  2023| 34|1.3| 2.8|0.2| 3.0|
|        6|  2024| 56|1.0| 8.0|2.4|36.8|
|        7|  2023| 22|2.9| 4.5|0.0|18.3|
|        8|  2023| 43|2.6| 3.4|0.8|13.0|
|        9|  2023| 76|2.5| 6.3|1.2|22.7|
|       10|  2023| 44|2.0| 1.5|0.9| 4.0|
|       11|  2023| 72|3.2| 3.1|1.0|15.4|
|       12|  2023| 77|3.3| 4.9|0.6|34.2|
|       13|  2023| 46|3.4| 9.2|1.6| 4.8|
|       14|  2023| 67|2.6|10.2|2.9| 2.2|
|       15|  2023| 71|2.4|10.5|2.1|13.0|
|       16|  2023| 19|2.8| 3.6|0.8|36.1|
|       17|  2023| 44|0.8|11.0|1.2| 0.0|
|       18|  2023| 22|2.8|11.4|1.9| 2.5|
|       19|  2024| 39|1.4| 0.0|2.9|19.6|
|       20|  2023| 64|1.7| 5.8|1.4| 6.7|
+---------+------+---+---+----+---+----+
only showing top

In [27]:
# Vérifie combien de joueurs n’ont aucune équipe.

In [28]:
players.show()

+---------+---------+--------+--------+---------+
|Player_ID|   Player|    Team|Position|   Salary|
+---------+---------+--------+--------+---------+
|        1| Player_1| Unknown|      SF|4190611.0|
|        2| Player_2|    Heat|      SG|3050658.0|
|        3| Player_3|    Suns|      SF|4519194.0|
|        4| Player_4|Clippers|      SF|7000660.0|
|        5| Player_5|   Spurs|      PF|2907846.0|
|        6| Player_6|  Lakers|      PG|2907870.0|
|        7| Player_7|    Nets|      SF|7200107.0|
|        8| Player_8|    Nets|       C|      0.0|
|        9| Player_9|Clippers|      SF|2585069.0|
|       10|Player_10|Warriors|       C|4287781.0|
|       11|Player_11|    Nets|      PG|2592909.0|
|       12|Player_12|   Bulls|      SF|2589914.0|
|       13|Player_13|Warriors|      SG|3689425.0|
|       14|Player_14|  Lakers|      SF|1255896.0|
|       15|Player_15| Unknown|      SG|      0.0|
|       16|Player_16|   Spurs|      PG|2467846.0|
|       17|Player_17|    Heat|      SF|1970079.0|


In [30]:
# Vérifie combien de joueurs n’ont aucune équipe.
players.filter(col('Team') == 'Unknown').count()

9912